# Collaboration Dataset
This notebooks extracts the authors and co-authors from every title of the past CIAA-conferences for further work on collaboration patterns

###### written by Nils Dyck, 04.10.2022

In [23]:
import urllib.request, urllib.error, urllib.parse

import re #for using regular expressions by finding the paper titles

from bs4 import BeautifulSoup

counter = 0 #for the right year

with open('Collaboration.csv', 'w') as f:
        f.write('Venue;Author;Co-Authors;Title;Abstract\n')

allURLs = ['https://link.springer.com/book/10.1007/978-3-031-07469-1','https://link.springer.com/book/10.1007/978-3-030-79121-6','https://link.springer.com/book/10.1007/978-3-030-23679-3','https://link.springer.com/book/10.1007/978-3-319-94812-6','https://link.springer.com/book/10.1007/978-3-319-60134-2','https://link.springer.com/book/10.1007/978-3-319-40946-7','https://link.springer.com/book/10.1007/978-3-319-22360-5']

In [24]:
url = 'https://link.springer.com/conference/wia'
response = urllib.request.urlopen(url)
webContent = response.read().decode('UTF-8')
sausage = BeautifulSoup(webContent, 'html.parser')

#get links from the conferences
links = sausage.find_all(href=re.compile('book'))
for i in range(len(links)):
    links[i] = str(links[i])
    found = re.search('"/book/.*?"', links[i]).group()
    found = found.replace('"','')
    found = 'https://link.springer.com' + found
    links[i] = found
links

['https://link.springer.com/book/10.1007/978-3-031-07469-1',
 'https://link.springer.com/book/10.1007/978-3-030-79121-6',
 'https://link.springer.com/book/10.1007/978-3-030-23679-3',
 'https://link.springer.com/book/10.1007/978-3-319-94812-6',
 'https://link.springer.com/book/10.1007/978-3-319-60134-2',
 'https://link.springer.com/book/10.1007/978-3-319-40946-7',
 'https://link.springer.com/book/10.1007/978-3-319-22360-5',
 'https://link.springer.com/book/10.1007/978-3-319-08846-4',
 'https://link.springer.com/book/10.1007/978-3-642-39274-0',
 'https://link.springer.com/book/10.1007/978-3-642-31606-7',
 'https://link.springer.com/book/10.1007/978-3-642-22256-6',
 'https://link.springer.com/book/10.1007/978-3-642-18098-9',
 'https://link.springer.com/book/10.1007/978-3-642-02979-0',
 'https://link.springer.com/book/10.1007/978-3-540-70844-5',
 'https://link.springer.com/book/10.1007/978-3-540-76336-9',
 'https://link.springer.com/book/10.1007/11812128',
 'https://link.springer.com/book/

In [25]:
del links[17]


In [26]:
for link in links:
    response = urllib.request.urlopen(link)
    webContent = response.read().decode('UTF-8')
    soup = BeautifulSoup(webContent, 'html.parser')#cook the soup for every conference page

    titles = soup.find_all(href = re.compile('/chapter/')) #collects titles
    print(len(titles))

    #create urls for paper-sites
    urls = []
    for j in range(1,len(titles)+1):
        urls.append(link+'_'+str(j))
    for k in range(len(urls)):
        urls[k] = urls[k].replace('book', 'chapter')
    #print(urls)

    paper_sites = []
    for l in range(len(urls)):
        response = urllib.request.urlopen(urls[l])
        htmlCode = response.read().decode('UTF-8')
        paper_sites.append(BeautifulSoup(htmlCode, 'html.parser'))
    #finding bibliographic information for every paper from the given conference
    for site in paper_sites:
        #finding authors and saving them in a list called 'author_list'
        authors = site.find_all(href = re.compile('#auth.*'))
        author_list = []
        for author in authors:
            for string in author.stripped_strings:
                #print(repr(string))
                author_list.append(string)
        print(len(author_list))
        co_authors = []
        for i in range(len(author_list)):
            co_authors.append([])
            for j in range(len(author_list)):
                if i != j:
                    co_authors[i].append(author_list[j])
        co_author_string = []
        for k in range(len(co_authors)):
            co_author_string.append('')
            for l in range(len(co_authors[k])):
                co_author_string[k]+= (co_authors[k][l]+', ')
            co_author_string[k] = co_author_string[k][:-2]
        #print(co_author_string)
        #finding the abstract and saving it in a string called 'abstract_string'
        abstract = site.find(class_ = 'c-article-section__content')
        abstract_string = abstract.find('p')
        #Cleaning up the abstract a little bit
        abstract_string = str(abstract_string).replace(";",".") #for the csv-file there must not be any semicola in the abstracts
        abstract_string = re.sub("<.*?>","",abstract_string) #no html-tag should survive this action.
        abstract_string = abstract_string.replace("\n","")
         #finding title
        title = site.title
        title = str(title)
        title = title.replace('\xa0', ' ')
        title = re.sub("<.*?>", "", title) #no html-tag should survive this action.
        title = title.replace(' | SpringerLink', '')

        #writing the output-file
        with open('Collaboration.csv', 'a') as f:
            for i in range(len(author_list)):
                f.write(str(counter)+';')
                f.write(author_list[i]+';')
                f.write(str(co_author_string[i])+';')#clean form
                #f.write(str(co_authors[i])+';')#list form
                f.write(title+';')
                f.write(abstract_string)
                f.write('\n')
    counter+=1
        



17
3
['Markus Holzer, Christian Rauch', 'Hermann Gruber, Christian Rauch', 'Hermann Gruber, Markus Holzer']
2
['Rogério Reis', 'Nelma Moreira']
3
['Hyunjoon Cheon, Yo-Sub Han', 'Sicheol Sung, Yo-Sub Han', 'Sicheol Sung, Hyunjoon Cheon']
2
['Martin Berglund', 'Brink van der Merwe']
2
['Lydia Rodriguez de la Nava', 'Florent Jacquemard']
3
['Maurice Herwig, Martin Lange', 'Florian Bruse, Martin Lange', 'Florian Bruse, Maurice Herwig']
3
['Richard Mörbitz, Heiko Vogler', 'Frank Drewes, Heiko Vogler', 'Frank Drewes, Richard Mörbitz']
1
['']
3
['Robert Mercaş, Daniel Reidenbach', 'Laura K. Hutchinson, Daniel Reidenbach', 'Laura K. Hutchinson, Robert Mercaş']
2
['Uwe Meyer', 'Martin Kutrib']
3
['Andreas Malcher, Matthias Wendlandt', 'Martin Kutrib, Matthias Wendlandt', 'Martin Kutrib, Andreas Malcher']
2
['Viktor Olejár', 'Michal Hospodár']
1
['']
2
['Alexander Okhotin', 'Olga Martynova']
2
['Kai Salomaa', 'Oliver A. S. Lyon']
4
['Fırat Kıyak, Meriç Üngör, A. C. Cem Say', 'Öykü Yılmaz, Meriç 

HTTPError: HTTP Error 404: Not Found